# Pronalaženje značajki

In [149]:
class FeatureExtractor:
           
    def extractFeatures(self, image):# returns Feature[] for one image
        
        features_all = []
        i=7
        while True:
            w = np.power(2,i)
            if image.res_grayscale_image.shape[0] < 2*w:
                break
            
            
            xy = process_image(image, w)
            features = process_features(xy, w, image)
            
            if len(features)==0:
                break
            
            features_all.extend(features)
            i=i+1;

        print "Feature count: "+str(len(features_all))
        return features_all   #, feature_map

    def extractFeaturesMulti(self, images):# returns Feature[] for set of images
        feature_list = []
        
        for image in images:
            features = self.extractFeatures(image)
            feature_list.extend(features);
        
        return feature_list;

## Pomoćne funkcije za pronalaženje značajki

In [150]:
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters


def symmetry_factor(scan_line, index, width, kernel):
    left = scan_line[index-width:index].astype(float)
    right = np.flipud(scan_line[index:index+width]).astype(float)
    diff = np.abs(left - right)
    return diff.dot(kernel) #+255- np.std(left)+255 - np.std(right)#np.average(diff) 

def scan_whole_line(whole_image, height_index=400, width = 100):
    scan_line = whole_image[height_index,:]
    X = range(width,len(scan_line)-width)
    kernel = cv2.getGaussianKernel(width*2,0)[width:]*2
    return np.squeeze(np.array([symmetry_factor(scan_line,index,width,kernel) for index in X]))

def unique_rows(a,orderBy):
    order = np.lexsort(orderBy.T)
    a = a[order]
    orderBy = orderBy[order]
    diff = np.diff(orderBy, axis=0)
    ui = np.ones(len(orderBy), 'bool')
    ui[1:] = (diff != 0).any(axis=1) 
    return a[ui]

def create_symmetry_map(image,width,verbose=False):
    print width
    img = np.zeros(image.shape[1] - width*2)
    for num in xrange(1,image.shape[0]):
        results = scan_whole_line(image,num,width)
        img = np.vstack((img,results))
    img = img.astype(np.float32)
    kernel_size= 11
    if verbose:
        plt.figure(figsize=(10,10))
        plt.imshow(img,cmap='gray')
        plt.show()
    img = cv2.GaussianBlur(img,sigmaX=0,ksize=(kernel_size,1)) 
    img = cv2.Sobel(img,cv2.CV_32F,2,0,ksize=kernel_size)
    padding = np.zeros((img.shape[0],width))
    
    if verbose:
        plt.figure(figsize=(10,10))
        plt.imshow(img,cmap='gray')

        plt.show()
    return np.hstack((padding,img,padding))


def process_image(image, w):

    processed_map_horizontal = create_symmetry_map(image.res_grayscale_image,w)

    processed_map_vertical = create_symmetry_map(image.res_grayscale_image.T,w)

    processed_map_horizontal[processed_map_horizontal<0]=0
    processed_map_vertical[processed_map_vertical<0]=0
    processed_map = processed_map_horizontal*processed_map_vertical.T
    processed_map = processed_map.astype(np.float32)
    processed_map[processed_map<np.max(processed_map)*0.0] = 0

    neighborhood_size = 5
    threshold = 0

    data = processed_map

    data_max = filters.maximum_filter(data, neighborhood_size)
    maxima = (data == data_max)
    data_min = filters.minimum_filter(data, neighborhood_size)
    diff = ((data_max - data_min) > threshold)
    maxima[diff == 0] = 0

    labeled, num_objects = ndimage.label(maxima)
    xy = np.array(ndimage.center_of_mass(data, labeled, range(1, num_objects+1)))

    return xy

def process_features(features, width, image):

    assert width in [np.power(2,i) for i in xrange(0,10)]
    feature_vectors = []

    for feature_y, feature_x in features:
        scan_line = image.res_grayscale_image[feature_y,:]
        index = feature_x

        left = scan_line[index-width:index].astype(float)
        right = np.flipud(scan_line[index:index+width]).astype(float)

        avg_line = (left + right)/2
        avg_value = np.average(avg_line)

        feature_vector = avg_line<avg_value

        feature_vector = sp.ndimage.zoom(feature_vector, 128/float(width), order=0).astype(int)

        feature_vectors.append(Feature(feature_vector, feature_x, feature_y,image, image.id_building, width))                            

    return feature_vectors


# Test area 51

In [151]:
import scipy as sp
import sklearn
import itertools
import cv2
import numpy as np

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [152]:
def pretty_show(img):
    plt.figure(figsize=(20,20))
    plt.imshow(img,cmap='gray')
    plt.show()

In [153]:
del preprocess_image
def preprocess_image(image, max_width=640, max_height = 480):
    factor = 1
    if image.shape[0]>max_height:
        factor = max_height/float(image.shape[0])
    if image.shape[1]>max_width:
        f2 = max_width/float(image.shape[1])
        factor = f2 if f2 <factor else factor
    resized = cv2.resize(image,(0,0),fx=factor,fy=factor,interpolation=cv2.INTER_CUBIC) if factor <1 else image
    gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
    return resized,gray

In [154]:
class Feature:
    def __init__(self, hashFeatures, x, y, image, id_building, w):
        self.hashFeatures = hashFeatures
        self.x = x
        self.y = y
        self.image = image
        self.id_building = id_building
        self.w = w
        
    def __str__(self):
        return str(self.hashFeatures)
    
class Image:
    def __init__(self, path, max_width, max_height, id_building):
        self.path = path
        self.image = cv2.imread(path)
        self.resized_image, self.res_grayscale_image = preprocess_image(self.image, max_width, max_height)
        self.id_building = id_building

In [155]:
train_image_paths = [
    'angle.jpg',
    'angle2.jpg',
    'angle3.jpg',
    'ne_notredame.jpg',
    'angle4.jpg',
    'angle5.jpg',
    'tor2.jpg',
    'tor3.jpg',
    'tor4.jpg'
]
images = []
j=1;
for i in train_image_paths:
    images.append(Image(i, 640, 480, j))
    j=j+1

In [156]:
fe = FeatureExtractor()

features_all_map = fe.extractFeaturesMulti(images)


128
128
Feature count: 341
128
128
Feature count: 244
128
128
Feature count: 299
128
128
Feature count: 340
128
128
Feature count: 321
128
128
Feature count: 346
128
128
Feature count: 345
Feature count: 0
128
128
Feature count: 583


C:\Users\Matea\Anaconda2\lib\site-packages\ipykernel\__main__.py:84: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Matea\Anaconda2\lib\site-packages\ipykernel\__main__.py:87: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Matea\Anaconda2\lib\site-packages\ipykernel\__main__.py:88: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
